# PyEO Forest Alerts: How to send forest alert reports to a list of users

This notebook was developed for pyeo on a Linux VM for Azure Labs.

- This notebook will send emails to user email addresses in a text file, informing them that new forest alerts are ready.
- The file name(s) of new vectorised forest alert report files will be included in the email.

# Setup: Requirements to use this Notebook

Look at the current directory path.

In [15]:
pwd

'/home/cmsstudent/Desktop/pyeo_data/36NYG'

Change to the pyeo home directory and define the config file path.

In [16]:
import os
pyeo_dir = '/home/cmsstudent/pyeo'
os.chdir(pyeo_dir)
workdir = os.getcwd()
print(workdir)
config_path = os.path.join(pyeo_dir, 'pyeo_linux_azure.ini')

/home/cmsstudent/pyeo


We did this in the previous notebook step-by-step. Here, we initialise the notebook in one code cell to speed up the process.

In [17]:
import argparse
import configparser
import cProfile
import datetime
import email
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.message import EmailMessage
import glob
import pandas as pd
import os
from osgeo import gdal
import shutil
import smtplib
import ssl
import sys
from pyeo import filesystem_utilities
from pyeo.filesystem_utilities import config_to_log
import warnings
import zipfile
from pyeo.acd_national import (acd_initialisation,
                                 acd_config_to_log,
                                 acd_roi_tile_intersection)

gdal.UseExceptions()
warnings.simplefilter("ignore", category=UserWarning)
config_dict, acd_log = acd_initialisation(config_path)
pyeo_dir = config_dict["pyeo_dir"]
os.chdir(pyeo_dir) # ensures pyeo is looking in the correct directory
config_to_log(config_dict, acd_log)
tilelist_filepath = acd_roi_tile_intersection(config_dict, acd_log)
# Build the folder structure for all tiles
tiles_to_process = pd.read_csv(tilelist_filepath)["tile"]
for tile_to_process in tiles_to_process:
    individual_tile_directory_path = os.path.join(config_dict["tile_dir"], tile_to_process)
    filesystem_utilities.create_folder_structure_for_tiles(individual_tile_directory_path)

# to process all tiles: tile_to_process = None 

# process only the first tile in the list:
tile_to_process = tiles_to_process[0]

# initialise the main log file
log_dir = os.path.join(pyeo_dir, config_dict["log_dir"], config_dict["log_filename"])
print(f"Log file: {log_dir}")
log = filesystem_utilities.init_log_acd(
    log_path=log_dir,
    logger_name=f"pyeo"
)

start_date = config_dict["start_date"]
end_date = config_dict["end_date"]
composite_start_date = config_dict["composite_start"]
composite_end_date = config_dict["composite_end"]
cloud_cover = config_dict["cloud_cover"]
cloud_certainty_threshold = config_dict["cloud_certainty_threshold"]
model_path = config_dict["model_path"]
sen2cor_path = config_dict["sen2cor_path"]
epsg = config_dict["epsg"]
bands = config_dict["bands"]
resolution = config_dict["resolution_string"]
out_resolution = config_dict["output_resolution"]
buffer_size = config_dict["buffer_size_cloud_masking"]
buffer_size_composite = config_dict["buffer_size_cloud_masking_composite"]
max_image_number = config_dict["download_limit"]
faulty_granule_threshold = config_dict["faulty_granule_threshold"]
download_limit = config_dict["download_limit"]

skip_existing = config_dict["do_skip_existing"]
sieve = config_dict["sieve"]
from_classes = config_dict["from_classes"]
to_classes = config_dict["to_classes"]

download_source = config_dict["download_source"]
if download_source == "scihub":
    log.info("scihub API is the download source")
if download_source == "dataspace":
    log.info("dataspace API is the download source")

os.chdir(config_dict["pyeo_dir"]) # ensures pyeo is looking in the correct directory
credentials_path = os.path.join(pyeo_dir, config_dict["credentials_path"])
if not os.path.isfile(credentials_path):
    log.error(f"The credentials path does not exist  :{credentials_path}")
    log.error(f"Current working directory :{os.getcwd()}")
    log.error("Exiting")
    sys.exit(1)

change_image_dir = os.path.join(individual_tile_directory_path, r"images")
l1_image_dir = os.path.join(individual_tile_directory_path, r"images", r"L1C")
l2_image_dir = os.path.join(individual_tile_directory_path, r"images", r"L2A")
l2_masked_image_dir = os.path.join(individual_tile_directory_path, r"images", r"cloud_masked")
categorised_image_dir = os.path.join(individual_tile_directory_path, r"output", r"classifications")
probability_image_dir = os.path.join(individual_tile_directory_path, r"output", r"probabilities")
reports_dir = os.path.join(individual_tile_directory_path, r"output", r"reports")
sieved_image_dir = os.path.join(individual_tile_directory_path, r"output", r"sieved")
composite_dir = os.path.join(individual_tile_directory_path, r"composite")
composite_l1_image_dir = os.path.join(individual_tile_directory_path, r"composite", r"L1C")
composite_l2_image_dir = os.path.join(individual_tile_directory_path, r"composite", r"L2A")
composite_l2_masked_image_dir = os.path.join(individual_tile_directory_path, r"composite", r"cloud_masked")
quicklook_dir = os.path.join(individual_tile_directory_path, r"output", r"quicklooks")
email_alerts = config_dict['email_alerts']
email_list_file = os.path.join(pyeo_dir, config_dict['email_list_file'])
whatsapp_alerts = config_dict['whatsapp_alerts']
tile_dir = config_dict["tile_dir"]

try:
    credentials_conf = configparser.ConfigParser(allow_no_value=True, interpolation=None)
    credentials_conf.read(credentials_path)
    credentials_dict = {}
    if email_alerts:
        log.info(f"Reading your email credentials from {credentials_path}")
        email_sender = credentials_conf["email"]["user"]
        email_app_password = credentials_conf["email"]["pass"]
    if whatsapp_alerts:
        log.info(f"Reading your WhatsApp credentials from {credentials_path}")
        whatsapp_sender = credentials_conf["whatsapp"]["user"]
        whatsapp_password = credentials_conf["whatsapp"]["pass"]
except:
    log.error(f"Could not open file or email/WhatsApp credentials missing: {credentials_path}")
    log.info("Create the file with your login credentials.")
    sys.exit(1)

if download_source == "dataspace":
    credentials_dict["sent_2"] = {}
    credentials_dict["sent_2"]["user"] = credentials_conf["dataspace"]["user"]
    credentials_dict["sent_2"]["pass"] = credentials_conf["dataspace"]["pass"]
    sen_user = credentials_dict["sent_2"]["user"]
    sen_pass = credentials_dict["sent_2"]["pass"]

if download_source == "scihub":
    credentials_dict["sent_2"] = {}
    credentials_dict["sent_2"]["user"] = credentials_conf["sent_2"]["user"]
    credentials_dict["sent_2"]["pass"] = credentials_conf["sent_2"]["pass"]
    sen_user = credentials_dict["sent_2"]["user"]
    sen_pass = credentials_dict["sent_2"]["pass"]    

2024-10-25 15:05:22,310: INFO: ---------------------------------------------------------------
2024-10-25 15:05:22,311: INFO: ---                 PROCESSING START                        ---
2024-10-25 15:05:22,311: INFO: ---------------------------------------------------------------
2024-10-25 15:05:22,312: INFO: conda environment path found: /home/cmsstudent/miniconda3//envs/pyeo_env
2024-10-25 15:05:22,313: INFO: True
2024-10-25 15:05:22,314: INFO: ---------------------------------------------------------------
2024-10-25 15:05:22,314: INFO: ---                  INTEGRATED PROCESSING START            ---
2024-10-25 15:05:22,315: INFO: ---------------------------------------------------------------
2024-10-25 15:05:22,315: INFO: Reading in parameters defined in: /home/cmsstudent/pyeo/pyeo_linux_azure.ini
2024-10-25 15:05:22,316: INFO: ---------------------------------------------------------------
2024-10-25 15:05:22,317: INFO: ----------------------------
2024-10-25 15:05:22,317: IN

Log file: /home/cmsstudent/Desktop/pyeo_data/log/my_log.log


# Send report
This notebook provides an app for sending out summary information on detected changes between
the vectorised change detection report images to users in various ways.

Vectorisation should be done as part of detect_change.py by setting in the ini file:

  do_vectorise = True

It supports only email alerts with a Gmail account at this stage. You will need to create an app password in your Gmail account under Security settings and save that into the credentials.ini file instead of your normal Gmail user password.

WhatsApp alerts will be added in the future, hopefully.

Shapefiles in the reports_dir will be zipped up to avoid sending the same file twice.

In [18]:
if config_dict["do_distribution"]: # if distribution of email alerts is switched on in the pyeo.ini file
    if email_alerts:
        try:
            elf = open(email_list_file, 'r')
            recipients = elf.readlines()
            for line in recipients:
                if "," not in line:
                    log.info(f"Dropping line without comma: {line}")
                    recipients.remove(line)
            log.info("Recipients of email alerts:")
            for line in recipients:
                name = line.split(",")[0]
                email_address = line.split(",")[1]
                log.info(f"{name}, {email_address}")
            elf.close()
        except:
            log.error(f"ABORTING. Email distribution list file could not be read: {email_list_file}.")
            sys.exit(1)            
        
    # start tile processing
    if tile_to_process == "None":
        # if no tile ID is given by the call to the function, use the geometry file
        #   to get the tile ID list
        tile_based_processing_override = False
        tilelist_filepath = acd_roi_tile_intersection(config_dict, log)
        tiles_to_process = list(pd.read_csv(tilelist_filepath)["tile"])

        # move filelist file from roi dir to main directory and save txt file
        tilelist_filepath = shutil.move(
            tilelist_filepath, 
            os.path.join(
                config_dict["tile_dir"], 
                tilelist_filepath.split(os.path.sep)[-1])
            )
        try:
            tilelist_txt_filepath = os.path.join(
                            config_dict["tile_dir"], 
                            tilelist_filepath.split(os.path.sep)[-1].split('.')[0]+'.txt'
                            )

            pd.DataFrame({"tile": tiles_to_process}).to_csv(
                tilelist_txt_filepath, 
                header=True, 
                index=False
            )
            log.info(f"Saved: {tilelist_txt_filepath}")
        except:
            log.error(f"Could not write to {tilelist_filepath}")

        log.info("Region of interest processing based on ROI file.")        

    else:
        # if a tile ID is specified, use that and do not use the tile intersection
        #   method to get the tile ID list
        tile_based_processing_override = True
        tiles_to_process = [tile_to_process]
        log.info("Tile based processing selected. Overriding the geometry file intersection method")
        log.info("  to get the list of tile IDs.")

    log.info(str(len(tiles_to_process)) + " Sentinel-2 tile report file(s) to process.")

2024-10-25 15:05:22,491: INFO: Dropping line without comma: 

2024-10-25 15:05:22,492: INFO: Recipients of email alerts:
2024-10-25 15:05:22,493: INFO: Heiko,  hb91@le.ac.uk

2024-10-25 15:05:22,494: INFO: Tile based processing selected. Overriding the geometry file intersection method
2024-10-25 15:05:22,495: INFO:   to get the list of tile IDs.
2024-10-25 15:05:22,495: INFO: 1 Sentinel-2 tile report file(s) to process.


Now let's iterate over the recipient emails and send all of them the forest alert email, if a new report shapefile is located in the outputs directory underneath the tile directory.

In [27]:
if config_dict["do_distribution"]: # if distribution of email alerts is switched on in the pyeo.ini file
    # iterate over the tiles
    for tile_to_process in tiles_to_process:
        log.info("Sending out the latest reports for Sentinel-2 tile: " + tile_to_process)
        individual_tile_directory_path = os.path.join(tile_dir, tile_to_process)
        log.info(individual_tile_directory_path)

        # create tile directory structure if not yet present
        try:
            filesystem_utilities.create_folder_structure_for_tiles(individual_tile_directory_path)
            probability_image_dir = os.path.join(individual_tile_directory_path, r"output", r"probabilities")
            reports_dir = os.path.join(individual_tile_directory_path, r"output", r"reports")
        except:
            log.error("ERROR: Tile subdirectory paths could not be created")
            sys.exit(1)

        # initialise tile log file
        tile_log_file = os.path.join(
            individual_tile_directory_path, 
            "log", 
            tile_to_process + ".log"
            )

        log.info(f"Redirecting log output to tile log: {tile_log_file}")
        tile_log = filesystem_utilities.init_log_acd(
            log_path=tile_log_file,
            logger_name="pyeo_"+tile_to_process
        )
        
        tile_log.info("---------------------------------------------------------------")
        tile_log.info(f"---  TILE PROCESSING START: {tile_to_process}                          ---")
        tile_log.info("---------------------------------------------------------------")
        tile_log.info(
            "Sending vectorised reports if available."
        )

        search_term = "report_*" + tile_to_process + "*.shp"

        tile_log.info(f"Searching for vectorised change report shapefiles in {reports_dir}")
        tile_log.info(f" containing: {search_term}.")

        vector_files = glob.glob(os.path.join(reports_dir, search_term))
        if len(vector_files) > 0:
            for vector_file in vector_files:
                tile_log.info(f"  {vector_file}")

            # zip up all the shapefiles and ancillary files
            zipped_vector_files = []
            for sf in vector_files:
                # split off the ".shp" file extension
                file_id = sf.split(".")[0]
                files_to_zip = glob.glob(file_id+".*")
                files_to_zip = [f for f in files_to_zip if not f.endswith('.zip')]
                #for z in files_to_zip:
                zipped_file = os.path.join(reports_dir, file_id + '.zip')
                with zipfile.ZipFile(
                    zipped_file, "w", compression=zipfile.ZIP_DEFLATED
                    ) as zf:
                        for f in files_to_zip:
                            zf.write(f, os.path.basename(f))
    
                if os.path.exists(zipped_file):
                    zipped_vector_files.append(zipped_file)
                    for f in files_to_zip:
                        os.remove(f)
                else:
                    tile_log.error(f"Zipping failed: {zipped_file}")
    
            tile_log.info(
                f"{len(zipped_vector_files)} report shapefiles found and zipped up."
            )

            if len(zipped_vector_files) == 0:
                tile_log.info("No new forest alert vector files found.")
                tile_log.info("No message will be sent.")
            else:
                if email_alerts:
                    elf = open(email_list_file, 'r')
                    recipients = elf.readlines()
                    for line in recipients:
                        if "," not in line:
                            log.info(f"Dropping line without comma: {line}")
                            recipients.remove(line)
                    
                    for r, recipient in enumerate(recipients):
                        # Remove the newline character
                        recipient_name = recipient.strip().split(",")[0]
                        recipient_email = recipient.strip().split(",")[1]
                        tile_log.info(
                            f"Sending email from {email_sender} to {recipient_name} " +
                            f"at {recipient_email}."
                            )
                        for f in zipped_vector_files:
                            file_size_mb = os.stat(f).st_size / (1024 * 1024)
                            start_date_dt = datetime.datetime.strptime(start_date, '%y%m%d')
                            if end_date = 'TODAY':
                                end_date_dt = datetime.date.today().strftime('%Y%m%d')
                            else:
                                end_date_dt = datetime.datetime.strptime(end_date, '%y%m%d')
                            body =  f"Dear {recipient_name},\n\n"+\
                               "New pyeo forest alerts have been detected.\n"+\
                               f"Time period: from {start_date_dt} to {end_date_dt}\n"+\
                               f"Vector file: {f}\n"+\
                               f"Zipped vector file size [MB]: {file_size_mb}\n\n"+\
                               "Please check the individual alerts and consider action "+\
                               "for those you want investigating.\n\n"+\
                               "Date of sending this email: "+\
                               f"{datetime.date.today().strftime('%Y%m%d')}\n\n"+\
                               "Best regards,\n"+\
                               "The pyeo forest alerts team\n"+\
                               "DISCLAIMER: The alerts are providing without any warranty.\n"+\
                               "IMPORTANT: Do not reply to this email."
            
                            subject_line = "New pyeo forest alerts are ready for you "+\
                                f"(Sentinel-2 tile {tile_to_process})"
                    
                            # Create a multipart message and set headers
                            message = MIMEMultipart()
                            message["From"] = email_sender
                            message["To"] = recipient_email
                            message["Subject"] = subject_line
                            message["Bcc"] = recipient_email  # Recommended for mass emails
                            
                            # Add body text to email message
                            message.attach(MIMEText(body, "plain"))

                            # Add attachment.
                            # Careful: Some mail servers block emails with zip file 
                            #   attachments
                            with open(f, "rb") as attachment:
                                # Add file as application/octet-stream
                                # Email client can usually download this automatically as attachment
                                part = MIMEBase("application", "octet-stream")
                                part.set_payload(attachment.read())

                            # Encode file in ASCII characters to send by email    
                            encoders.encode_base64(part)

                            # Add header as key/value pair to attachment part
                            part.add_header(
                                "Content-Disposition",
                                f"attachment; filename= {os.path.basename(f)}",
                            )

                            # Add attachment to message and convert message to string
                            message.attach(part)
                            text = message.as_string()

                            # Log in to server using secure context and send email
                            context = ssl.create_default_context()
                            with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
                                server.login(email_sender, email_app_password)
                                server.sendmail(email_sender, recipient_email, text)
                            
                    tile_log.info(" ")
                    tile_log.info("Info on vectorised reports has been emailed to the contact list.")
                    tile_log.info(" ")
    
                if whatsapp_alerts and len(vector_files)>0:
                    tile_log.error("WhatsApp alerts have not been implemented yet.")
                    #TODO: WhatsApp
                    # run a separate script in a different Python environment using pywhatkit
                    # os.script("path to bash file")
                    # The bash files needs to do the following:
                    #   make sure WhatsApp is open and running
                    #   conda activate whatsapp_env
                    #   python send_whatsapp.py
    		
                    '''        
                    tile_log.info("---------------------------------------------------------------")
                    tile_log.info("Info on vectorised reports has been sent via WhatsApp to the contact list.")
                    tile_log.info("---------------------------------------------------------------")
                    tile_log.info(" ")
                    '''        
        else:
            tile_log.warning(f"No new matching vector files found for tile: {tile_to_process}")
            tile_log.info("No message will be sent.")

        tile_log.info("---------------------------------------------------------------")
        tile_log.info("---             TILE PROCESSING END                           ---")
        tile_log.info("---------------------------------------------------------------")


2024-10-25 15:12:11,264: INFO: Sending out the latest reports for Sentinel-2 tile: 36NXG
2024-10-25 15:12:11,266: INFO: /home/cmsstudent/Desktop/pyeo_data/36NXG
2024-10-25 15:12:11,267: INFO: Redirecting log output to tile log: /home/cmsstudent/Desktop/pyeo_data/36NXG/log/36NXG.log
2024-10-25 15:12:11,268: INFO: ---------------------------------------------------------------
2024-10-25 15:12:11,269: INFO: ---                 PROCESSING START                        ---
2024-10-25 15:12:11,270: INFO: ---------------------------------------------------------------
2024-10-25 15:12:11,271: INFO: ---------------------------------------------------------------
2024-10-25 15:12:11,272: INFO: ---  TILE PROCESSING START: 36NXG                          ---
2024-10-25 15:12:11,273: INFO: ---------------------------------------------------------------
2024-10-25 15:12:11,274: INFO: Sending vectorised reports if available.
2024-10-25 15:12:11,275: INFO: Searching for vectorised change report shapef